In [38]:
import os

In [39]:
from bs4 import BeautifulSoup


In [40]:
%pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [41]:
%pip install playwright


Note: you may need to restart the kernel to use updated packages.


In [42]:
os.system('playwright install')

0

In [43]:
from playwright.async_api  import async_playwright, TimeoutError as PlaywrightTimeoutError
import time

In [44]:
%pip install playwright

Note: you may need to restart the kernel to use updated packages.


In [45]:
!playwright install

In [46]:
SEASONS = list(range(2023,2025))

In [47]:
SEASONS

[2023, 2024]

In [48]:
DATA_DIR = '/Users/sumedhmarathe/Documents/GitHub/NBA_Predictions/data'
STANDINGS_DIR = os.path.join(DATA_DIR, 'standings')
SCORES_DIR = os.path.join(DATA_DIR, 'scores')


In [49]:
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(STANDINGS_DIR, exist_ok=True)
os.makedirs(SCORES_DIR, exist_ok=True)

In [50]:
%pip show playwright


Name: playwright
Version: 1.47.0
Summary: A high-level API to automate web browsers
Home-page: https://github.com/Microsoft/playwright-python
Author: Microsoft Corporation
Author-email: 
License: Apache-2.0
Location: /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages
Requires: greenlet, pyee
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [51]:
!pip show playwright

Name: playwright
Version: 1.47.0
Summary: A high-level API to automate web browsers
Home-page: https://github.com/Microsoft/playwright-python
Author: Microsoft Corporation
Author-email: 
License: Apache-2.0
Location: /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages
Requires: greenlet, pyee
Required-by: 


In [52]:
async def get_html(url, selector,sleep=5, retries=3):
  html = None
  for i in range(1, retries+1):
    (time.sleep(sleep*i))

    try:
      async with async_playwright() as p:
        browser = await p.chromium.launch()
        page = await browser.new_page()
        await page.goto(url)
        print(await page.title())
        html = await page.inner_html(selector)
    except PlaywrightTimeoutError:
      print('TimeoutError on {url}')
      continue
    else:
      break

  return html

In [53]:
async def scrape_season(season):
  url = f'https://www.basketball-reference.com/leagues/NBA_{season}_games.html'
  html = await get_html(url, '#content .filter')
  
  soup = BeautifulSoup(html)
  links = soup.find_all('a')
  hrefs = [l["href"] for l in links]
  standings_pages = [f"https://basketball-reference.com{l}" for l in hrefs]

  for url in standings_pages:
    save_path = os.path.join(STANDINGS_DIR, url.split('/')[-1])
    if os.path.exists(save_path):
      continue
    html = await get_html(url, "#all_schedule")
    with open(save_path,'w+') as f:
      f.write(html)

In [54]:
for season in SEASONS:
  await scrape_season(season)

TimeoutError on {url}
TimeoutError on {url}
2022-23 NBA Schedule | Basketball-Reference.com
2023-24 NBA Schedule | Basketball-Reference.com
2023-24 NBA Schedule | Basketball-Reference.com
2023-24 NBA Schedule | Basketball-Reference.com
TimeoutError on {url}
2023-24 NBA Schedule | Basketball-Reference.com
2023-24 NBA Schedule | Basketball-Reference.com
2023-24 NBA Schedule | Basketball-Reference.com
2023-24 NBA Schedule | Basketball-Reference.com
2023-24 NBA Schedule | Basketball-Reference.com
2023-24 NBA Schedule | Basketball-Reference.com
2023-24 NBA Schedule | Basketball-Reference.com


In [55]:
standings_pages = os.listdir(STANDINGS_DIR)